## 最基础的反向传播自动求导数




其实有许多内含知识，比如叶子节点是手动设置为requires_grad为true 或false ，正向传播时会自动将各个变量设置为true  



In [32]:
import torch

In [33]:
torch.cuda.is_available()

True

In [34]:
X_DATA=torch.tensor([[1.0],[2.0],[3.0]])

y_data=torch.tensor([[2.0],[4.0],[6.0]])

w = torch.Tensor([1.0])
w.requires_grad = True

tensor([1.], requires_grad=True)

In [35]:
def forward(x):
    return x*w

def loss(x,y):
    return(y-forward(x))**2

for i in range(100):
    for x,y in zip(X_DATA,y_data):
        l=loss(x,y)
        l.backward()#每次结束都会释放计算图
        print('\tgrad:',x,y,w.grad.item()) #item是直接取出来变成标量
        w.data=w.data - 0.01*w.grad.data#注意！！！！！！！使用data并不会构建计算图
        
        w.grad.data.zero_()#对梯度清零，不然是累加额
        
    print("loss:",i,l.item())

## torch实现线性回归


实现基本是以构造类的方式来进行构建模型，同时要继承nn.module这个父类。一定要实现_init_和forward  

官网连接 linear https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html  
  
`def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()`

In [59]:
X_DATA=torch.tensor([[1.0],[2.0],[3.0]])

y_data=torch.tensor([[2.0],[4.0],[6.0]])

class LinearModel(torch.nn.Module):
    def __init__(self):
        super(LinearModel,self).__init__()
        self.linear = torch.nn.Linear(1,1)#linear类是线性变换类，可以自动构建内含的weight
        
    def forward(self,x):
        y_pre = self.linear(x)
        return y_pre

model = LinearModel()
criterion = torch.nn.MSELoss(size_average=False)#构造损失函数
optimizer =torch.optim.SGD(model.parameters(),lr=0.01)#sgd随机梯度下降优化器

  
        
for epoch in range (1000):
    y_pred = model(X_DATA)#forward
    loss = criterion(y_pred,y_data)#计算损失
    print(epoch,loss.item())
    
    optimizer.zero_grad()#反向传播前清零
    loss.backward()
    optimizer.step()#更新一波
    
    
print('w=',model.linear.weight.item())


0 24.854263305664062
1 11.079386711120605
2 4.946984767913818
3 2.216801166534424
4 1.0011929273605347
5 0.4598323106765747
6 0.21863098442554474
7 0.11105507612228394
8 0.06296786665916443
9 0.041366614401340485
10 0.03155869245529175
11 0.027003534138202667
12 0.02478955127298832
13 0.023620355874300003
14 0.022919120267033577
15 0.022428546100854874
16 0.022034531459212303
17 0.021686004474759102
18 0.02136007323861122
19 0.021046752110123634
20 0.02074146270751953
21 0.020442185923457146
22 0.020147819072008133
23 0.019857969135046005
24 0.019572483375668526
25 0.01929115131497383
26 0.019013911485671997
27 0.018740596249699593
28 0.018471261486411095
29 0.018205834552645683
30 0.017944157123565674
31 0.017686231061816216
32 0.01743210107088089
33 0.017181582748889923
34 0.01693468913435936
35 0.016691232100129128
36 0.016451396048069
37 0.016214944422245026
38 0.01598195917904377
39 0.015752308070659637
40 0.01552583184093237
41 0.015302746556699276
42 0.015082815662026405
43 0.01

902 5.947102010850358e-08
903 5.860916019173601e-08
904 5.7774528272602765e-08
905 5.683519077592791e-08
906 5.6123340641534014e-08
907 5.530678492959851e-08
908 5.4496254620062246e-08
909 5.36050634991625e-08
910 5.293287586027873e-08
911 5.2139967010589316e-08
912 5.1419206670288986e-08
913 5.0656865369091975e-08
914 4.994633684418659e-08
915 4.9240398425354215e-08
916 4.843769829676603e-08
917 4.776148898599786e-08
918 4.7090281896089436e-08
919 4.6424077027040767e-08
920 4.578714651870541e-08
921 4.5130263970349915e-08
922 4.4521257791529933e-08
923 4.3958209516858915e-08
924 4.321765345594031e-08
925 4.2639513253561745e-08
926 4.2047588522109436e-08
927 4.151894472670392e-08
928 4.085781313278858e-08
929 4.0319392269339005e-08
930 3.970838236000418e-08
931 3.919468838375906e-08
932 3.8592318674091075e-08
933 3.8085900655460136e-08
934 3.7492171145459e-08
935 3.700968420616846e-08
936 3.6463291053223656e-08
937 3.588257868614164e-08
938 3.543244986303762e-08
939 3.491442157610436e-

这里真的很妙，首先module的底层是实现了call函数，是会调用forward函数。  
当我们构建的模块被调用时，会首先调用我们自己写的forward函数，而我们自己的forward函数中会去调用linear模型的forward函数  

` def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight, self.bias)`  
        就是个矩阵乘法
        
### 总结一下过程

1 先算y_pre，然后计算loss（forward)  

2 backward 然后进行更新

In [60]:
model(torch.tensor([4.0]))

tensor([8.0001], grad_fn=<AddBackward0>)